In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import re

In [5]:
HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "sec-ch-ua": '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
#     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    "User-Agent": "Mozilla/5.0"
}

In [6]:
## 페이지 번호 쿼리 작성
def get_page_num(user,passwd,host,db):
    engine = create_engine(f'postgresql://{user}:{passwd}@{host}/{db}')
    engine.connect()

    query = """
    SELECT stock_code, page_num from crawling_docs where check_num = 0;
    """
    
    df=pd.read_sql(query, con=engine)
    codes, page = df.values[0]
    return codes, page


In [7]:

def NS_users_crawler(codes, start, term):
    
    result_df = pd.DataFrame([])
    current_page = start
    ## 시작 페이지부터 한번에 수집할 페이지 텀만큼 반복 수행
    for page in range(start+1, start+1+term):
        
        url = "https://finance.naver.com/item/board.naver?code=%s&page=%s" %(codes, str(page))    
        html = requests.get(url, headers=HEADERS).content
        soup = BeautifulSoup(html.decode('euc-kr', 'replace'), 'html.parser')
        table = soup.find('table', {'class' : 'type2'})
        tb = table.select('tbody > tr')
        
        ## 마지막 페이지+1으로 조회해도 마지막 페이지와 동일함
        s= tb[5].select('a')[0]['href']
        real_page_num = re.findall(r'page=(\d{1,9})', s) ## 실제 페이지 번호
        
        if str(page) != real_page_num[0]: return result_df, current_page, True ## 페이지 번호가 다를 경우 중지.
        
        ## 게시글 하나씩 result_df에 저장
        for i in range(2, len(tb)):
            if len(tb[i].select('td > span')) > 0:
                date = tb[i].select('td > span')[0].text
                title = tb[i].select('td.title > a')[0]['title']
                views = tb[i].select('td > span')[1].text
                pos = tb[i].select('td > strong')[0].text
                neg = tb[i].select('td > strong')[1].text
                table = pd.DataFrame({'code' : codes, '날짜':[date], '제목' : [title], '조회': [views], '공감' : [pos], '비공감' : [neg]})
                result_df = pd.concat([result_df,table], axis=0)
        current_page = page
                
    ## 완료시 df, page, check_sum 반환   
    return result_df, current_page, False

In [8]:
codes, page = get_page_num()
data, page, checkSum = NS_users_crawler(codes, page, term = 10)
data

,code,날짜,제목,조회,공감,비공감
0,000020,2023.05.04 13:36,사골국 ♤☆☆,78,1,0
0,000020,2023.05.04 13:31,큰기대 ♤☆☆,89,4,0
0,000020,2023.05.04 13:28,경보제약 상간다,130,2,1
0,000020,2023.05.04 13:24,현실즉시,97,4,1
0,000020,2023.05.04 10:45,배당금,216,0,7
...,...,...,...,...,...,...
0,000020,2023.04.04 00:23,이게 뭔 소리냐?,1010,9,0
0,000020,2023.04.03 23:02,그지같?종목,274,3,0
0,000020,2023.04.03 20:38,동화는 명품... 재벌집 막내 아들이 아니고..?,334,1,6
0,000020,2023.04.03 10:28,5000주 매수한사람인데,1107,5,5
